In [45]:
import pandas as pd
import pyodbc
import numpy as np
from pandas import ExcelWriter
import datetime
start_ = datetime.datetime.today()
scraper='HARRINGTON'

In [46]:
fecha = str(datetime.date.today())
fecha

'2020-02-11'

In [48]:
fecha = str(datetime.date.today())
ipath = f'./Salida/Harrington2020-02-11.xlsx'
df_harrington = pd.read_excel(ipath)

In [49]:
server = 'tcp:192.168.1.6'
database ="planeamiento"
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+
                      ';DATABASE='+database+
                      ';UID='+username+
                      ';PWD='+ password) 

In [50]:
maxCorrida = "select max(id_sc3_corrida)+1 from sc3_detalle where origen ='HARRINGTON UY' "
maxCorrida = pd.read_sql(maxCorrida,cnxn) 

In [51]:
maxCorrida = int(maxCorrida[""][0])

In [52]:
TIPOLOGIA = ['Remera','Camisa','Pantalón','Pantalon','Bermuda','Campera','Jean','Short','T-shirt','Sweater','Chaleco','Cardigan',
             'Sobretodo','Saco','Traje','Montgomery']

TIPOLOGIA = list(map(lambda x: x.upper(),TIPOLOGIA))

In [53]:
df_harrington['Tipo'] = 0

In [54]:
df_harrington.loc[df_harrington['Descripcion']=='T','Descripcion'] = "T-Shirt A Rayas Harrington Label"
df_harrington.loc[df_harrington['Descripcion']=='T-Shirt A Rayas Harrington Label','Color'] = "Gris/blanco"
#'T-Shirt A Rayas Harrington Label'
#https://www.harrington.com.uy/catalogo/t-shirt-a-rayas-harrington-label-gris-blanco_520709_000217

In [55]:
for index, row in df_harrington.iterrows():
    for TIPO in TIPOLOGIA:
        if row['Descripcion'].upper().find(TIPO) != -1 :
            df_harrington.loc[index,'Tipo'] = TIPO


In [58]:
for index, row in df_harrington.iterrows():
    if row['Tipo'] =='PANTALÓN':
        df_harrington.loc[index,'Tipo'] = 'PANTALON'
    if row['Descripcion'].find('Panatlon') != -1:
        df_harrington.loc[index,'Tipo'] = 'PANTALON'

In [59]:
df_harrington["Precio Nuevo"] = [precio.split('$U')[2].strip() if len(precio.split('$U'))==3 else precio.split('$U')[1] for precio in df_harrington["Precio"]]
df_harrington["Precio Anterior"] = [precio.split('$U')[1].strip() for precio in df_harrington["Precio"]]

In [60]:
df_harrington['Precio Nuevo'] = (df_harrington["Precio Nuevo"]
                         .str.extract(r"([\d,\.]+)", expand=False)
                         .str.replace(".", "")
                         .astype(float))


In [61]:
df_harrington['Precio Anterior'] = (df_harrington["Precio Anterior"]
                         .str.extract(r"([\d,\.]+)", expand=False)
                         .str.replace(".", "")
                         .astype(float))


In [62]:
df_harrington['TIPO_AUX'] = df_harrington['Tipo']
df_harrington['COLOR_AUX'] = df_harrington['Color']

In [63]:
if len(df_harrington[df_harrington['Tipo']==0]) != 0:
    
    print(f"REVISAR {scraper} condicion >> (Tipo == 0 > 0) ")
    raise 

In [64]:
df_harrington[df_harrington['Tipo'] == 0]

,id Producto,Descripcion,Color,Precio,Url Producto,Url Imagen,Fecha,Marca,Moneda,Sexo,Origen,Tipo,Precio Nuevo,Precio Anterior,TIPO_AUX,COLOR_AUX


In [65]:
df_harrington['Origen'] ='HARRINGTON UY'

In [66]:
cursor = cnxn.cursor()
for index,row in df_harrington.iterrows():                                                            
    cursor.execute("INSERT INTO sc3_detalle([id_sc3_producto],[id_sc3_corrida],[marca],[tipo],[tipo_es],[color],[color_es],[sexo],[descripcion],[moneda],[precio],[precio_original],[fecha_alta],[url_imagen],[url_producto],[origen]) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                   row['id Producto'], 
                   maxCorrida,
                   row['Marca'],
                   row['Tipo'],
                   row['TIPO_AUX'],
                   row['Color'],
                   row['COLOR_AUX'],
                   row['Sexo'],
                   row['Descripcion'], 
                   row['Moneda'],
                   row['Precio Nuevo'], 
                   row['Precio Anterior'], 
                   row['Fecha'],
                   row['Url Imagen'], 
                   row['Url Producto'],
                   row['Origen']) 
    cnxn.commit()
cursor.close()
cnxn.close()

In [67]:
end_ = datetime.datetime.today()

print(f'Inserts {scraper} en : {end_-start_}')

Inserts HARRINGTON en : 0:02:38.488561


In [1]:
#!jupyter nbconvert --to script 'Harrington_insert.ipynb'

[NbConvertApp] Converting notebook Harrington_insert.ipynb to script
[NbConvertApp] Writing 4090 bytes to Harrington_insert.py
